<a class="anchor" id="5"></a>
#### Content
* [Prepare required files for generating the multiplex network](#1)
    * [gene sets (e.g., GO-BP)](#2)
    * [processed scRNAseq dataset with distinct populations of cells (e.g., PBMC)](#3)
        
* [Run the demo function to quantify the similarities among gene sets via the KNN-based similarity measure](#4)

In [2]:
from src import demo
import numpy as np
from gsea_api.molecular_signatures_db import GeneSets
import scanpy as sc
import json
import anndata

<a class="anchor" id="1"></a>
I. Inputs for building a multiplex newtork of gene sets (via KNN similarity measure):\
     $\qquad$ a. gene sets (e.g., GO biology process)\
     $\qquad$ b. a scRNAseq dataset (e.g., PBMC3000) that consists of several cell populations


<a class="anchor" id="2"></a>

In [3]:
'''
The contents in BP.json, BP.TERM.json files include gene sets' IDs, names, constituent genes, description information 
(retrieved from gseGO/GO.db). 
'''
with open("BP.json") as f:
    BP:dict = json.load(f)
with open("BP.TERM.json") as f:
    BP_term:dict = json.load(f)

In [4]:
GO_BP_ID = list(BP.keys())
GO_BP_gs_name = BP_term 
GO_BP_gene_set = list(BP.values())
print(f"GO_BP database has {len(GO_BP_gene_set )} gene sets.")

GO_BP database has 16029 gene sets.


In [5]:
gene_set_dict:dict = {'gene_set_gene_symbols':GO_BP_gene_set, 'gene_set_annotation':GO_BP_gs_name,\
                 'gene_set_collection_name':'GO-BP'}

In [5]:
'''
Alternatively, the GO-BP dataset (e.g., .gmt file) can be downloaded from Broad Insitute or other resources 
and loaded by the Python package gsea_api. 
Notice that,
    a. the order of the loaded gene sets is not deterministic, 
which may cause errors in the downstream analysis if being reckless of the indices of the gene sets.
    b. The GO dataset's content from Broad Institute differs from the above due to different selection criteria. 
'''
GO_BP_gene_set = GeneSets.from_gmt('c5.go.bp.v2023.1.Hs.symbols.gmt')
GO_BP_gs_name = [item.name for item in GO_BP_gene_set.gene_sets]
GO_BP_gene_set = [list(GO_BP_gene_set.gene_sets[i].genes) for i in range(len(GO_BP_gene_set.gene_sets))]
print(f"GO-BP database has {len(GO_BP_gene_set )} gene sets.")

FileNotFoundError: [Errno 2] No such file or directory: 'c5.go.bp.v2023.1.Hs.symbols.gmt'

[Back](#5)
$~$
<a class="anchor" id="3"></a>

In [6]:
### processed single-cell log-count dataset
scRNAseq:anndata.AnnData = sc.read_h5ad("pbmc3k.h5ad")

In [7]:
np.unique(scRNAseq.obs['cell.type'], return_counts = True)

(array(['B', 'CD14+ Mono', 'CD8 T', 'DC', 'FCGR3A+ Mono', 'Memory CD4 T',
        'NK', 'Naive CD4 T', 'Platelet'], dtype=object),
 array([344, 480, 271,  32, 162, 483, 155, 697,  14]))

In [8]:
### filter cell types with a small sample size
cell_label = ['B', 'CD14+ Mono', 'CD8 T','Memory CD4 T', 'Naive CD4 T']

In [9]:
processed_scRNAseq_obj_lst:list = [scRNAseq[(scRNAseq.obs['cell.type'] == cell)] for cell in cell_label]

$~$

II. Input the created files (gene_set_dict, processed_scRNAseq_obj_lst) to the demo function:gene_set_similarity_multiplex_network
will return a multiplex network object (large) with intermediate files stored in a folder 'GO_BP' whose name can be arbitrary.

The process takes a long time (2 hours) depending on the sample size, number of gene sets, cores, CPU capability, memory size, hyperparameters in the KNN model (e.g., k), etc.
<a class="anchor" id="4"></a>

In [10]:
'''
We filter gene sets with their number of sequenced genes outside the range of 10 to 2000. 
Those bounds can be adjusted customarily. If more than 90% of cells in a gene-set feature space have all-zero counts,
this gene set will also be filtered since obtaining a meaningful similarity based on too-sparse signals is difficult.

The implemented normalized KNN similarity measure uses $k = \sqrt{N}$, 
"auto" algorithm, and L2 metric as the default setting. 
'''
nt = demo.gene_set_similarity_multiplex_network(processed_scRNAseq_obj_lst, population_labels = cell_label,\
                  num_core = 30, gene_set_dict = gene_set_dict, output_folder = 'GO_BP')

Population B has been instantiated.


100%|████████████████████████████████████████████████████████████████████████████| 16029/16029 [00:26<00:00, 599.95it/s]


5336 over 16029 pass the filteration!
Finding k-nearest neighbors of gene sets...


100%|███████████████████████████████████████████████████████████████████████████████| 5336/5336 [03:09<00:00, 28.14it/s]

Computing the similarity matrix...


...

100%|████████████████████████████████████████████████████████████████████████████| 16029/16029 [00:46<00:00, 347.03it/s]


5336 over 16029 pass the filteration!
Finding k-nearest neighbors of gene sets...


100%|███████████████████████████████████████████████████████████████████████████████| 5336/5336 [03:39<00:00, 24.31it/s]

Computing the similarity matrix...


Complete!
The pipeline for Naive CD4 T finishes!


In [ ]:
'''
We filter gene sets with their number of sequenced genes outside the range of 10 to 2000. 
Those bounds can be adjusted customarily. If more than 90% of cells in a gene-set feature space have all-zero counts,
this gene set will also be filtered since obtaining a meaningful similarity based on too-sparse signals is difficult.

The implemented normalized KNN similarity measure uses $k = \sqrt{N}$, 
"auto" algorithm, and L2 metric as the default setting. 
'''
nt = demo.gene_set_similarity_multiplex_network(processed_scRNAseq_obj_lst, population_labels = cell_label,\
                  num_core = 10, gene_set_dict = gene_set_dict, output_folder = 'GO_BP')

Population B has been instantiated.


100%|████████████████████████████████████| 16029/16029 [01:02<00:00, 257.55it/s]


5336 over 16029 pass the filteration!
Finding k-nearest neighbors of gene sets...


100%|███████████████████████████████████████| 5336/5336 [04:07<00:00, 21.52it/s]

Computing the similarity matrix...


[Back](#5)
$~$

In [11]:
'''
With those intermediate files (tissue objects), we can build the multiplex network 
when needed rather than storing it as a large file.
Ensure the 'GO_BP' folder is a subfolder of the current working directory. 
Or cd into 'GO_BP' and remove the folder argument in the following statement.
'''

# nt = demo.tissue_obj_multiplex_network(collection = "GO-BP", labels = None, folder = 'GO_BP')